In [1]:
'''

Created Date: Tue Nov 15 2023

Revised Date: Sat Jul 21 2025

Author: Qiming Ding, Yukun Zhang

Description: CHD Openfenmion hamiltonian and qiskit hamiltonian


Copyright (c) 2025

'''
# Standard library imports
import time
import pickle
# Scientific computing and data handling libraries
import numpy as np
import pandas as pd
import datetime
import itertools
from itertools import combinations, product
import functools
from functools import reduce
from joblib import Memory, Parallel, delayed
#np.set_printoptions(suppress=True, precision=6, threshold=np.inf)
# OpenFermion imports
import openfermion as of
import openfermion.ops.representations as reps
from openfermion.config import EQ_TOLERANCE
from openfermion.transforms import jordan_wigner, get_fermion_operator
from openfermion.linalg import get_ground_state, get_sparse_operator

# PySCF imports
import pyscf
from pyscf import gto, scf, ao2mo, ci, cc, fci, mp, M, mcscf
from pyscf.tools import molden

In [2]:
def get_active_space_integrals(h1e,
                                h2e,
                                occupied_indices=None,
                                active_indices=None):
    """Restricts a molecule at a spatial orbital level to an active space
    This active space may be defined by a list of active indices and
        doubly occupied indices. Note that one_body_integrals and
        two_body_integrals must be defined
        n an orthonormal basis set.  
    Args:
        one_body_integrals (ndarray): One-electron integrals over spatial orbitals.
        two_body_integrals (ndarray): Two-electron integrals over spatial orbitals.
        occupied_indices (list): A list of spatial orbital indices indicating which orbitals should be considered doubly occupied.
        active_indices (list): A list of spatial orbital indices indicating which orbitals should be considered active.

    Returns:
        tuple: Tuple with the following entries:
        - core_constant (float): Adjustment to constant shift in Hamiltonian from integrating out core orbitals.
        - one_body_integrals_new (ndarray): One-electron integrals over active space.
        - two_body_integrals_new (ndarray): Two-electron integrals over active space.
    """
    # Fix data type for a few edge cases
    occupied_indices = [] if occupied_indices is None else occupied_indices
    if (len(active_indices) < 1):
        raise ValueError('Some active indices required for reduction.')

    # Get integrals.
    one_body_integrals, two_body_integrals = h1e, h2e
    return reps.get_active_space_integrals(one_body_integrals,
                                            two_body_integrals,
                                            occupied_indices, active_indices)
    
def spinorb_from_spatial(one_body_integrals, two_body_integrals):
    n_qubits = 2 * one_body_integrals.shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros(
        (n_qubits, n_qubits, n_qubits, n_qubits))
    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[p, q] = one_body_integrals[p, q]
            one_body_coefficients[p + n_qubits // 2, q + n_qubits // 2] = one_body_integrals[p, q]

            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[p, q + n_qubits // 2, r + n_qubits // 2, s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[p + n_qubits // 2,  q, r, s +
                                          n_qubits // 2] = (two_body_integrals[p, q, r, s])

                    # Same spin
                    two_body_coefficients[p, q, r, s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[p + n_qubits // 2, q + n_qubits // 2, r + n_qubits // 2, s +
                                          n_qubits // 2] = (two_body_integrals[p, q, r, s])
    # Truncate.
    one_body_coefficients[
        np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.
    two_body_coefficients[
        np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.

    return one_body_coefficients, two_body_coefficients

In [3]:
def generate_hamiltonian(atoms, d, occupied_indices, active_indices, 
                          n_elec=None, use_cas=False, basis='sto-3g'):
    """
    计算分子的哈密顿量，并可选择使用CASSCF或全空间方法。

    Args:
        atoms (list): PySCF格式的原子定义列表，例如 [('H', (0,0,0)), ('H', (0,0,0.74))]。
        d (float): 与原子坐标相关的距离参数，用于生成动态文件名。
        occupied_indices (list): 冻结的核心轨道指数。
        active_indices (list): 活性空间轨道指数。
        n_elec (int, optional): 活性空间中的电子数。如果为 None，则自动推断。
        use_cas (bool, optional): 如果为 True，则使用 CASSCF 方法计算积分。
                                  否则，使用 RHF 轨道进行全空间计算。默认为 False。
        basis (str, optional): 使用的基组名称。默认为 'sto-3g'。

    Returns:
        tuple: 包含分子哈密顿量、量子比特哈密顿量、FCI能量等多个计算结果。
    """
    # --- 1. 分子定义与RHF计算 ---
    spin = 0
    charge = 0
    # 使用函数参数'basis'来定义分子
    mol = gto.M(atom=atoms, basis=basis, spin=spin, charge=charge,
                unit='Angstrom', verbose=1)
    
    hf = scf.RHF(mol)
    hf.conv_tol = 1e-9
    hf.max_cycle = 20000
    hf.kernel()

    for j in range(10):
        mo = hf.stability()[0]
        if np.allclose(mo, hf.mo_coeff):
            print(f"RHF converged and is stable after {j} cycles.")
            break
        dm = hf.make_rdm1(mo, hf.mo_occ)
        hf = hf.run(dm)
    else: # for-else 结构，如果循环正常结束（未被break），则执行
        print("Warning: RHF stability not fully reached after 10 cycles.")

    fci_calc = fci.FCI(mol, hf.mo_coeff)
    FCI_val = fci_calc.kernel()[0]
    print(f'Full CI (FCI) energy: {FCI_val:.8f}')

    atom_str = '-'.join([a[0] for a in atoms]) # e.g., 'C-H-D'
    safe_basis_str = basis.replace('*', 's').replace('/', '_')

    if use_cas:
        print("\nUsing CASSCF to generate integrals for the active space...")
        mycas = mcscf.CASSCF(hf, len(active_indices), n_elec)
        mycas.kernel() # 执行CASSCF计算以优化轨道
        
        h1e, constant = mycas.get_h1cas()
        h2e = mycas.get_h2cas()
        h2e = pyscf.ao2mo.restore('1', h2e, len(active_indices))
        
        # 定义动态文件名
        filename = f'{atom_str}_{safe_basis_str}_d{d:.2f}_cas.molden'
        print(f"Saving CASSCF orbitals to {filename}")
        with open(filename, 'w') as f1:
            molden.header(mol, f1)
            molden.orbital_coeff(mol, f1, mycas.mo_coeff, ene=mycas.mo_energy, occ=mycas.mo_occ)

    else:
        print("\nUsing full space RHF orbitals to generate integrals...")
        h1e_ao = mol.intor("int1e_kin") + mol.intor("int1e_nuc")
        h2e_ao = mol.intor("int2e")
        
        scf_c = hf.mo_coeff
        constant = mol.energy_nuc()

        h1e = scf_c.T @ h1e_ao @ scf_c
        h2e = pyscf.ao2mo.incore.full(h2e_ao, scf_c)

        filename = f'{atom_str}_{safe_basis_str}_d{d:.2f}_full_rhf.molden'
        print(f"Saving RHF orbitals to {filename}")
        with open(filename, 'w') as f1:
            molden.header(mol, f1)
            molden.orbital_coeff(mol, f1, hf.mo_coeff, ene=hf.mo_energy, occ=hf.mo_occ)

    h2e = h2e.transpose(0, 2, 3, 1)

    print("\nProjecting integrals into the specified active space...")
    if occupied_indices is not None and active_indices is not None:
        core_adjustment, one_body_integrals, two_body_integrals = get_active_space_integrals(
            h1e, h2e, occupied_indices, active_indices)
        constant += core_adjustment
    else:
        one_body_integrals = h1e
        two_body_integrals = h2e

    one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_integrals, two_body_integrals)
    
    molecular_hamiltonian = reps.InteractionOperator(constant, one_body_coefficients, 0.5 * two_body_coefficients)
    
    qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
    qubit_hamiltonian.compress()
    
    sparse_hamiltonian = get_sparse_operator(qubit_hamiltonian)
    precise_diagonalization_energy, state = get_ground_state(sparse_hamiltonian)
    
    print("\n--- Final Results ---")
    print(f'Final constant part of Hamiltonian: {constant:.8f}')
    print(f'Ground state energy from diagonalization: {precise_diagonalization_energy:.8f} Hartree.')
    print(f'Benchmark FCI energy: {FCI_val:.8f} Hartree.')
    print(f'Difference (Diagonalization - FCI): {precise_diagonalization_energy - FCI_val:.8f} Hartree.')
    
    return (molecular_hamiltonian, qubit_hamiltonian, FCI_val, one_body_coefficients, 
            two_body_coefficients, constant, n_elec, precise_diagonalization_energy)

In [4]:
def create_H_n_molecule(num_hydrogen_atoms, d):
    # 构建分子结构字符串
    molecule = ''
    for i in range(num_hydrogen_atoms):
        # 计算当前氢原子的位置
        position = i * d
        # 将氢原子添加到分子字符串中
        molecule += f'H .0 .0 {position}; '

    # 移除最后一个分号和空格
    molecule = molecule.strip('; ')
    
    return molecule

def design_molecule(molecule, d):
    if molecule.startswith("H"):
        num = int(''.join(filter(str.isdigit, molecule)))
        atoms = create_H_n_molecule(num, d)
        occupied_indices = None
        active_indices = None
        n_elec = num
        use_cas = False
    if molecule == "LiH":
        atoms = f'H .0, .0, .0; Li .0, .0, {d}'
        occupied_indices = [0]
        active_indices = [1, 2, 5]
        n_elec = 2      
        num_spin_orbitals = 6
        n_particles = 2
        # num_particles = [1, 1]  # [alpha electrons, beta electrons]
        num_qubits = 6
        use_cas = False
    elif molecule == "F2":
        atoms = f'F .0, .0, .0; F .0, .0, {d}'
        occupied_indices=[0,1,2,3]
        active_indices=[4,5,6,7,8,9]
        n_elec = 10
        num_spin_orbitals = 12
        num_particles = [9, 9]  # [alpha electrons, beta electrons]
        num_qubits = 12
        use_cas = False
    elif molecule == "H4":
        atoms = f'H .0, .0, .0; H .0, .0, {d}; H .0, .0, {2*d}; H .0, .0, {3*d}'
        occupied_indices = None
        active_indices = None
        n_elec = 4
        use_cas = False
    elif molecule == "H4s":
        atoms = f'H .0, .0, .0; H .0, .0, {d}; H .0, {d}, .0; H .0, {d}, {d}'
        occupied_indices = None
        active_indices = None
        n_elec = 4
        use_cas = False
    elif molecule == "N2":
        atoms = f'N .0, .0, .0; N .0, .0, {d}'
        occupied_indices = [0, 1, 2, 3]
        active_indices = [4, 5, 6, 7, 8, 9]
        n_elec = 6
        use_cas = False
    return molecule, atoms, occupied_indices, active_indices, n_elec, use_cas

In [5]:
if __name__ == "__main__":
    molecule = "H2"
    d_list = []
    current_value = 0.4

    while current_value <= 1.2:
        d_list.append(round(current_value, 1))
        current_value += 0.1

    while current_value <= 4.5:
        d_list.append(round(current_value, 1))
        current_value += 0.3

    print(d_list)
    
    for d in d_list:
        molecule, atoms, occupied_indices, active_indices, n_elec, use_cas = design_molecule(molecule, d)
        molecular_hamiltonian, qubit_hamiltonian, FCI_val, one_body_coefficients, two_body_coefficients, constant,  n_elec, Precise_diagonalization_energy = generate_hamiltonian(atoms,d, occupied_indices, active_indices, n_elec, use_cas)
        n_particles = n_elec
        print(FCI_val)
        print(Precise_diagonalization_energy)
        num_qubits = np.size(molecular_hamiltonian.one_body_tensor, 0) 
        data_to_save = {
            'molecular_hamiltonian': molecular_hamiltonian,
            'qubit_hamiltonian': qubit_hamiltonian,
            'FCI_val': FCI_val,
            'one_body_coefficients': one_body_coefficients,
            'two_body_coefficients': two_body_coefficients,
            'constant': constant,
            'n_elec': n_elec,
            'n_particles': n_particles,
            'num_qubits': num_qubits,
            'Precise_diagonalization_energy':Precise_diagonalization_energy}
    
        print(n_elec)
        print(num_qubits)
        # 文件名
        filename = f"{molecule}_{d}_sto-3g.pkl"

        # 使用pickle保存字典
        with open(filename, 'wb') as file:
            pickle.dump(data_to_save, file)
        print(f"所有变量已存储为 {filename}")

[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4.0, 4.3]
RHF converged and is stable after 0 cycles.
Full CI (FCI) energy: -0.91414970

Using full space RHF orbitals to generate integrals...
Saving RHF orbitals to H- -.-0- -.-0- -0-.-0-;- -H- -.-0- -.-0- -0-.-4_sto-3g_d0.40_full_rhf.molden

Projecting integrals into the specified active space...

--- Final Results ---
Final constant part of Hamiltonian: 1.32294303
Ground state energy from diagonalization: -0.91414970 Hartree.
Benchmark FCI energy: -0.91414970 Hartree.
Difference (Diagonalization - FCI): 0.00000000 Hartree.
-0.9141497046270852
-0.914149704627083
2
4
所有变量已存储为 H2_0.4_sto-3g.pkl
RHF converged and is stable after 0 cycles.
Full CI (FCI) energy: -1.05515979

Using full space RHF orbitals to generate integrals...
Saving RHF orbitals to H- -.-0- -.-0- -0-.-0-;- -H- -.-0- -.-0- -0-.-5_sto-3g_d0.50_full_rhf.molden

Projecting integrals into the specified active space...

--- Final Resu